In [1]:
# Algoritmo base
##### Este é o arquivo
# https://www.cs.utexas.edu/~flame/Notes/NotesOnCholReal.pdf

# Alterações feitas:

# A matriz L = zero(A) está sendo criada fora da função
# As alterações estão sendo feitas na própria matriz A
# Utilizar .= e .-=, operações in-place, otimizaram a alocação

In [2]:
using(LinearAlgebra)

In [262]:
##### Nível 2

# Algoritmo do livro (applied numerical linear algebra)
function  Cholesky_factorize(A, L)
    # L: matriz de zeros que será alterada, mesmo tamanho de A
    
    for j in 1:size(A, 1)

        L[j, j] = (A[j, j] - sum(L[j, 1:j-1].^2))^(.5)

        for i in (j+1):size(A, 1)
        
            L[i, j] = (A[i, j] - dot(L[i, 1:j-1], L[j, 1:j-1]))/L[j, j]

        end

    end

    return L
end

Cholesky_factorize (generic function with 1 method)

In [263]:
### Nível 3 ######

function Cholesky_by_blocks(A, L, block_size)

    n = size(A, 1)
    
    for j in 1:block_size:n
        b = min(n - j + 1, block_size)
        # Realizando as alterações na própria matriz A, poupa alocações
        A[j:(j+b-1), j:(j+b-1)] .= Cholesky_factorize(A[j:(j + b - 1), j:(j + b - 1)], L[j:(j + b - 1), j:(j + b - 1)])
        A[(j+b):n, j:( j + b - 1)] .= A[(j+b):n, j:(j + b -1)]/A[j:(j + b - 1), j:(j + b - 1)]'
        A[(j+b):n, (j + b):n] .-= A[(j+b):n, j:(j + b - 1)]*A[(j+b):n, j:(j + b - 1)]'

    end

    return tril(A)
end

Cholesky_by_blocks (generic function with 2 methods)

In [272]:
n = 800
size_block = 512

X = randn(n, n)
A = X*X'
B = copy(A)

@time cholesky(A)
@time Cholesky_factorize(A, zero(A))
@time Cholesky_by_blocks(A, zero(A), size_block)
print("- - - - - - - - --  - - - - - - - - - - -- - ")

  0.005367 seconds (5 allocations: 4.883 MiB)
  0.409153 seconds (640.80 k allocations: 1.359 GiB, 15.82% gc time)
  0.175711 seconds (345.93 k allocations: 468.582 MiB, 12.47% gc time)
- - - - - - - - --  - - - - - - - - - - -- - 

In [276]:
n = 400
size_block = 100

X = randn(n, n)
A = X*X'
B = copy(A) 

L1 = cholesky(A)
L2 = Cholesky_factorize(A, zero(A))
# A foi alterado
L3 = Cholesky_by_blocks(A, zero(A), size_block)

display(norm(B - L1.L*L1.U))
display(norm(B - L2*L2'))
display(norm(B - L3*L3'))

2.429502041402825e-12

2.2874668820722864e-12

2.033198073797186e-12

In [210]:

L1 = cholesky(B).L
L2 = Cholesky_factorize(B, zero(A))
L3 = Cholesky_by_blocks(B, zero(A), 2)

display(L1)
display(L2)
display(L3)

400×400 LowerTriangular{Float64, Matrix{Float64}}:
 18.5769        ⋅           ⋅         …    ⋅         ⋅         ⋅ 
  2.7349      20.4128       ⋅              ⋅         ⋅         ⋅ 
  0.284417     0.212716   20.9617          ⋅         ⋅         ⋅ 
 -0.314962    -0.241034    0.0634597       ⋅         ⋅         ⋅ 
 -0.53611     -1.06239     0.910011        ⋅         ⋅         ⋅ 
 -0.622202     0.637129    1.34384    …    ⋅         ⋅         ⋅ 
 -0.0954134    0.695407    0.209896        ⋅         ⋅         ⋅ 
  1.44143      0.493231   -1.05663         ⋅         ⋅         ⋅ 
  0.656793     0.200772    2.30322         ⋅         ⋅         ⋅ 
 -0.987724    -0.42772     1.05562         ⋅         ⋅         ⋅ 
 -1.13081      0.573346    0.717995   …    ⋅         ⋅         ⋅ 
  1.40342      0.058292   -1.2518          ⋅         ⋅         ⋅ 
 -0.28648     -0.61088    -0.61886         ⋅         ⋅         ⋅ 
  ⋮                                   ⋱                       
  0.58149     -0.666549    1

400×400 Matrix{Float64}:
 18.5769       0.0         0.0        …   0.0       0.0       0.0
  2.7349      20.4128      0.0            0.0       0.0       0.0
  0.284417     0.212716   20.9617         0.0       0.0       0.0
 -0.314962    -0.241034    0.0634597      0.0       0.0       0.0
 -0.53611     -1.06239     0.910011       0.0       0.0       0.0
 -0.622202     0.637129    1.34384    …   0.0       0.0       0.0
 -0.0954134    0.695407    0.209896       0.0       0.0       0.0
  1.44143      0.493231   -1.05663        0.0       0.0       0.0
  0.656793     0.200772    2.30322        0.0       0.0       0.0
 -0.987724    -0.42772     1.05562        0.0       0.0       0.0
 -1.13081      0.573346    0.717995   …   0.0       0.0       0.0
  1.40342      0.058292   -1.2518         0.0       0.0       0.0
 -0.28648     -0.61088    -0.61886        0.0       0.0       0.0
  ⋮                                   ⋱                       
  0.58149     -0.666549    1.33229        0.0       0.

400×400 Matrix{Float64}:
 18.5769       0.0         0.0        …   0.0       0.0       0.0
  2.7349      20.4128      0.0            0.0       0.0       0.0
  0.284417     0.212716   20.9617         0.0       0.0       0.0
 -0.314962    -0.241034    0.0634597      0.0       0.0       0.0
 -0.53611     -1.06239     0.910011       0.0       0.0       0.0
 -0.622202     0.637129    1.34384    …   0.0       0.0       0.0
 -0.0954134    0.695407    0.209896       0.0       0.0       0.0
  1.44143      0.493231   -1.05663        0.0       0.0       0.0
  0.656793     0.200772    2.30322        0.0       0.0       0.0
 -0.987724    -0.42772     1.05562        0.0       0.0       0.0
 -1.13081      0.573346    0.717995   …   0.0       0.0       0.0
  1.40342      0.058292   -1.2518         0.0       0.0       0.0
 -0.28648     -0.61088    -0.61886        0.0       0.0       0.0
  ⋮                                   ⋱                       
  0.58149     -0.666549    1.33229        0.0       0.

In [138]:
x = [1 ; 2]

sum(x.*x)

5

In [218]:
?cholesky

search: cholesky cholesky! Cholesky CholeskyPivoted Cholesky_factorize



```
cholesky(A, Val(false); check = true) -> Cholesky
```

Compute the Cholesky factorization of a dense symmetric positive definite matrix `A` and return a [`Cholesky`](@ref) factorization. The matrix `A` can either be a [`Symmetric`](@ref) or [`Hermitian`](@ref) [`StridedMatrix`](@ref) or a *perfectly* symmetric or Hermitian `StridedMatrix`. The triangular Cholesky factor can be obtained from the factorization `F` with: `F.L` and `F.U`. The following functions are available for `Cholesky` objects: [`size`](@ref), [`\`](@ref), [`inv`](@ref), [`det`](@ref), [`logdet`](@ref) and [`isposdef`](@ref).

If you have a matrix `A` that is slightly non-Hermitian due to roundoff errors in its construction, wrap it in `Hermitian(A)` before passing it to `cholesky` in order to treat it as perfectly Hermitian.

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

# Examples

```jldoctest
julia> A = [4. 12. -16.; 12. 37. -43.; -16. -43. 98.]
3×3 Matrix{Float64}:
   4.0   12.0  -16.0
  12.0   37.0  -43.0
 -16.0  -43.0   98.0

julia> C = cholesky(A)
Cholesky{Float64, Matrix{Float64}}
U factor:
3×3 UpperTriangular{Float64, Matrix{Float64}}:
 2.0  6.0  -8.0
  ⋅   1.0   5.0
  ⋅    ⋅    3.0

julia> C.U
3×3 UpperTriangular{Float64, Matrix{Float64}}:
 2.0  6.0  -8.0
  ⋅   1.0   5.0
  ⋅    ⋅    3.0

julia> C.L
3×3 LowerTriangular{Float64, Matrix{Float64}}:
  2.0   ⋅    ⋅
  6.0  1.0   ⋅
 -8.0  5.0  3.0

julia> C.L * C.U == A
true
```

---

```
cholesky(A, Val(true); tol = 0.0, check = true) -> CholeskyPivoted
```

Compute the pivoted Cholesky factorization of a dense symmetric positive semi-definite matrix `A` and return a [`CholeskyPivoted`](@ref) factorization. The matrix `A` can either be a [`Symmetric`](@ref) or [`Hermitian`](@ref) [`StridedMatrix`](@ref) or a *perfectly* symmetric or Hermitian `StridedMatrix`. The triangular Cholesky factor can be obtained from the factorization `F` with: `F.L` and `F.U`. The following functions are available for `CholeskyPivoted` objects: [`size`](@ref), [`\`](@ref), [`inv`](@ref), [`det`](@ref), and [`rank`](@ref). The argument `tol` determines the tolerance for determining the rank. For negative values, the tolerance is the machine precision.

If you have a matrix `A` that is slightly non-Hermitian due to roundoff errors in its construction, wrap it in `Hermitian(A)` before passing it to `cholesky` in order to treat it as perfectly Hermitian.

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

---

```
cholesky(A::SparseMatrixCSC; shift = 0.0, check = true, perm = nothing) -> CHOLMOD.Factor
```

Compute the Cholesky factorization of a sparse positive definite matrix `A`. `A` must be a [`SparseMatrixCSC`](@ref) or a [`Symmetric`](@ref)/[`Hermitian`](@ref) view of a `SparseMatrixCSC`. Note that even if `A` doesn't have the type tag, it must still be symmetric or Hermitian. If `perm` is not given, a fill-reducing permutation is used. `F = cholesky(A)` is most frequently used to solve systems of equations with `F\b`, but also the methods [`diag`](@ref), [`det`](@ref), and [`logdet`](@ref) are defined for `F`. You can also extract individual factors from `F`, using `F.L`. However, since pivoting is on by default, the factorization is internally represented as `A == P'*L*L'*P` with a permutation matrix `P`; using just `L` without accounting for `P` will give incorrect answers. To include the effects of permutation, it's typically preferable to extract "combined" factors like `PtL = F.PtL` (the equivalent of `P'*L`) and `LtP = F.UP` (the equivalent of `L'*P`).

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

Setting the optional `shift` keyword argument computes the factorization of `A+shift*I` instead of `A`. If the `perm` argument is provided, it should be a permutation of `1:size(A,1)` giving the ordering to use (instead of CHOLMOD's default AMD ordering).

# Examples

In the following example, the fill-reducing permutation used is `[3, 2, 1]`. If `perm` is set to `1:3` to enforce no permutation, the number of nonzero elements in the factor is 6.

```jldoctest
julia> A = [2 1 1; 1 2 0; 1 0 2]
3×3 Matrix{Int64}:
 2  1  1
 1  2  0
 1  0  2

julia> C = cholesky(sparse(A))
SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  simplicial
maxnnz:  5
nnz:     5
success: true

julia> C.p
3-element Vector{Int64}:
 3
 2
 1

julia> L = sparse(C.L);

julia> Matrix(L)
3×3 Matrix{Float64}:
 1.41421   0.0       0.0
 0.0       1.41421   0.0
 0.707107  0.707107  1.0

julia> L * L' ≈ A[C.p, C.p]
true

julia> P = sparse(1:3, C.p, ones(3))
3×3 SparseMatrixCSC{Float64, Int64} with 3 stored entries:
  ⋅    ⋅   1.0
  ⋅   1.0   ⋅
 1.0   ⋅    ⋅

julia> P' * L * L' * P ≈ A
true

julia> C = cholesky(sparse(A), perm=1:3)
SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  simplicial
maxnnz:  6
nnz:     6
success: true

julia> L = sparse(C.L);

julia> Matrix(L)
3×3 Matrix{Float64}:
 1.41421    0.0       0.0
 0.707107   1.22474   0.0
 0.707107  -0.408248  1.1547

julia> L * L' ≈ A
true
```

!!! note
    This method uses the CHOLMOD library from SuiteSparse, which only supports doubles or complex doubles. Input matrices not of those element types will be converted to `SparseMatrixCSC{Float64}` or `SparseMatrixCSC{ComplexF64}` as appropriate.

    Many other functions from CHOLMOD are wrapped but not exported from the `Base.SparseArrays.CHOLMOD` module.

